In [1]:
import numpy as np

from numpy import polynomial
from numpy.polynomial.polynomial import Polynomial
from numpy.polynomial.polynomial import polyval

pol = Polynomial([1,2,3])

pol(1)

6.0

Dado um polinomio $p(x) = 4 - 4x^2 + 3x^3$, podemos obter o valor do do polinomio no ponto x utilizando a função `polyval`

In [2]:
pol2 = Polynomial([4, 0, -4, 3])
print(pol2)

for x in range(-3, 3):
    print(x, pol2(x))

poly([ 4.  0. -4.  3.])
-3 -113.0
-2 -36.0
-1 -3.0
0 4.0
1 3.0
2 12.0


In [3]:
pol2.linspace(6, domain=[-3,2])

(array([-3., -2., -1.,  0.,  1.,  2.]),
 array([-113.,  -36.,   -3.,    4.,    3.,   12.]))

### Trabalhando com matriz no numpy


In [4]:
matriz = np.matrix([[1,2,5],
           [1,4,3],
           [1,2,8]])

matriz

matrix([[1, 2, 5],
        [1, 4, 3],
        [1, 2, 8]])

In [5]:
linhas, colunas = matriz.shape
regioes_x, regioes_y = (colunas - 1), (linhas - 1)

print(linhas, colunas)

3 3


In [6]:
polinomio_size = 16 * (linhas - 1) * (colunas - 1)

matriz[2,0], matriz[2,1], matriz[2,2]

(1, 2, 8)

In [7]:
def line_edge(i , matriz):
    linhas, _ = matriz.shape
    return i == 0 or i == linhas - 1


def column_edge(i, matriz):
    _, colunas = matriz.shape
    return i == 0 or i == colunas - 1

[line_edge(x, matriz) and column_edge(y, matriz) for x in range(linhas) for y in range(colunas)]


[True, False, True, False, False, False, True, False, True]

## Polinomio

Criando o polinomio no eixo de $x$ e $y$ e aplicando a derivada

In [8]:
x = 1
y = 2

x1y2 = [ x**i * y**j for i in range(4) for j in range(4)]
x1y2

[1, 2, 4, 8, 1, 2, 4, 8, 1, 2, 4, 8, 1, 2, 4, 8]

In [9]:
def fx(x, y):
    return [ x**i * y**j for i in range(4) for j in range(4)]

def dx(x, y):
    return [ i*  x**(i-1) * y**j if i > 0 else 0 for i in range(4) for j in range(4)]


def dy(x, y):
    return [ x**i * j * y**(j-1) if j > 0 else 0 for i in range(4) for j in range(4)]


def dxdy(x, y):
    return [ i * x**(i-1) * j * y**(j-1) if (i > 0 and j > 0) else 0 for i in range(4) for j in range(4)]


def dxx(x, y):
    return [ (i * (i-1) * x**(i-2)) * (y**j) if i > 1 else 0 for i in range(4) for j in range(4)]


def dyy(x, y):
    return [ (x**i) * (j * (j-1) * y**(j-2)) if j > 1 else 0 for i in range(4) for j in range(4)]


Vamos verificar agora como fazer o reshape do vetor e coloca-lo na matriz

O objetivo aqui é colocar uma lista dentro de uma lista já existente. E por fim, excluir o excedente.


exemplo com array normal

In [10]:
ma1 = np.arange(4)
print(ma1)

ma2 = ma1.tolist()[:1] + [9,9] + ma1.tolist()[1:]
print(ma2)
ma2 = ma2[:4]
print(ma2)


[0 1 2 3]
[0, 9, 9, 1, 2, 3]
[0, 9, 9, 1]


exemplo utilizando o ndarray

In [11]:
ma1 = np.arange(4)
print(ma1)
print(type(ma1))

to_insert = np.array([9,9])
print(to_insert)
print(type(to_insert))

ma1.put(range(1,3), to_insert)
print(ma1)


[0 1 2 3]
<class 'numpy.ndarray'>
[9 9]
<class 'numpy.ndarray'>
[0 9 9 3]


## Trabalhando com o algoritmo

Agora vou tentar replicar toda a solução em casa.
Abaixo tentarei percorres todos os pontos e pegas as sua caracteristica, regiao e polinomio


In [12]:
def getRange(coluna, linha):
    inicio_range = 16 * ((linhas-1) * coluna + linha)
    return range(inicio_range, inicio_range + 16)

In [13]:
print(getRange(0,0))
print(getRange(0,1))
print(getRange(1,0))
print(getRange(1,1))

range(0, 16)
range(16, 32)
range(32, 48)
range(48, 64)


### Funções auxiliares

Vou colocar funções auxiliares para a construção do polinomio principal. Essas funções transformarao as regras em uma grande matriz de $16*linhas*colunas$

In [14]:
def getLineFromList(itens):
    line = np.zeros(polinomio_size)

    for (f, x, y, *posicoes, positivo) in itens:
        if(len(posicoes)==0):
            coluna, linha = (x - 1 if x > 0 else 0, y - 1 if y > 0 else 0)
        elif(len(posicoes)==2):
            coluna, linha = posicoes
        else:
            raise Exception('Wrong number of arguments. Expected 4 or 6 but received %d' % (len(item)))

        range_para_inserir = getRange(coluna, linha)        
        pol = np.array(f(x,y)) if positivo else np.array(f(x,y)) * -1
    
        line.put(range_para_inserir, pol)
    return line

In [15]:
def getLineFromTuple(item):
    line = np.zeros(polinomio_size)

    f, x, y, *posicoes, positivo = item

    if(len(posicoes)==0):
        coluna, linha = (x - 1 if x > 0 else 0, y - 1 if y > 0 else 0)
    elif(len(posicoes)==2):
        coluna, linha = posicoes
    else:
        raise Exception('Wrong number of arguments. Expected 4 or 6 but received %d' % (len(item)))

    range_para_inserir = getRange(coluna, linha)
    
    pol = np.array(f(x,y)) if positivo else np.array(f(x,y)) * -1

    line.put(range_para_inserir, pol)
    return line

In [16]:
x = 0
y = 1

coluna = x - 1 if x > 0 else 0
linha = y - 1 if y > 0 else 0

l = (fx, x, y, coluna, linha + 1, True)
getLineFromTuple(l)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

## Vértice

Tratando todas as 4 linhas criadas pelos vértice

In [17]:
def vertice(x, y):
    coluna, linha = (x - 1 if x > 0 else 0, y - 1 if y > 0 else 0)

    range_para_inserir = getRange(coluna, linha)

    funcoes_vertices = [
        (fx, x, y, True),
        (dxx, x, y, True),
        (dyy, x, y, True),
        (dxdy, x, y, True),
    ]

    pols = np.array([])
    for item in funcoes_vertices:
        pols = np.append(pols, getLineFromTuple(item))

    return np.reshape(pols, (-1, polinomio_size))


In [18]:
vertice(0,0)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [19]:
vertice(2,0)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  2.,  0.,  0.,
         0.,  4.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  2.,  0.,  0.,  0., 12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  4.,
         0.,  0.,  0.,  8.,  0.,  0.,  0., 16.,  0.,  0.,  

In [20]:
vertice(2,2)

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   1.,   2.,   4.,   8.,   2.,   4.,   8.,
         16.,   4.,   8.,  16.,  32.,   8.,  16.,  32.,  64.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   2.,   4.,   8.,  16.,  12.,  24.,  48.,  96.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.

## Aresta

In [21]:
def aresta_x(x, y):
    coluna, linha = (x - 1 if x > 0 else 0, y - 1 if y > 0 else 0)
    range_para_inserir = getRange(coluna, linha)

    funcoes_arestas = [
        (fx, x, y, True),
        (dxdy, x, y, True),
        (fx, x, y, coluna + 1, linha, True),
        [(dx, x, y, True),(dx, x, y, coluna + 1, linha, False)],
        [(dxx, x, y, True),(dxx, x, y, coluna + 1, linha, False)],
        (dx, x, y, True),
        (dxx, x, y, coluna + 1, linha, True),
        (dxdy, x, y, coluna + 1, linha, True),
    ]

    pols = np.array([])
    for item in funcoes_arestas:
        if isinstance(item, tuple):
            pols = np.append(pols, getLineFromTuple(item))
        elif isinstance(item, list):
            pols = np.append(pols, getLineFromList(item))

    return np.reshape(pols, (-1, polinomio_size))


In [22]:
dxx(1,2)

[0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 8, 16, 6, 12, 24, 48]

### Arestas no eixo y

In [23]:
def aresta_y(x, y):
    coluna, linha = (x - 1 if x > 0 else 0, y - 1 if y > 0 else 0)
    range_para_inserir = getRange(coluna, linha)

    funcoes_arestas = [
        (fx, x, y, True),
        (dxdy, x, y, True),
        (fx, x, y, coluna, linha + 1,True),
        [(dy, x, y, True),(dy, x, y, coluna, linha + 1, False)],
        [(dyy, x, y, True),(dyy, x, y, coluna, linha + 1, False)],
        (dx, x, y, True),
        (dxx, x, y, coluna, linha + 1, True),
        (dxdy, x, y, coluna, linha + 1, True),
    ]

    pols = np.array([])
    for item in funcoes_arestas:
        if isinstance(item, tuple):
            pols = np.append(pols, getLineFromTuple(item))
        elif isinstance(item, list):
            pols = np.append(pols, getLineFromList(item))
    
    return np.reshape(pols, (-1, polinomio_size))

In [24]:
x = 1
y = 2

r = aresta_y(x, y)
print(r[4])

[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   2.  12.   0.   0.   2.  12.   0.   0.   2.  12.
   0.   0.   2.  12.   0.   0.  -2. -12.   0.   0.  -2. -12.   0.   0.
  -2. -12.   0.   0.  -2. -12.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.]


## Pontos Internos

In [38]:
def interno(x, y):
    coluna, linha = (x - 1 if x > 0 else 0, y - 1 if y > 0 else 0)
    range_para_inserir = getRange(coluna, linha)

    funcoes_interna = [
        # f(x) de todas as regioes
        (fx, x, y, True),
        (fx, x, y, coluna + 1, linha, True),
        (fx, x, y, coluna, linha + 1, True),
        (fx, x, y, coluna + 1, linha + 1, True),
        # dxdy(x) de todas as regioes
        (dxdy, x, y, True),
        (dxdy, x, y, coluna + 1, linha, True),
        (dxdy, x, y, coluna, linha + 1, True),
        (dxdy, x, y, coluna + 1, linha + 1, True),
        # dx de todas as regioes em 2 linhas
        [(dx, x, y, True),(dx, x, y, coluna, linha + 1, False)],
        [(dx, x, y, coluna + 1, linha, True),(dx, x, y, coluna + 1, linha + 1, False)],
        # dy de todas as regioes em 2 linhas
        [(dy, x, y, True),(dy, x, y, coluna, linha + 1, False)],
        [(dy, x, y, coluna, linha + 1, True),(dy, x, y, coluna + 1, linha + 1, False)],
        # dxx de todas as regioes em 2 linhas
        [(dxx, x, y, True),(dxx, x, y, coluna, linha + 1, False)],
        [(dxx, x, y, coluna + 1, linha, True),(dxx, x, y, coluna + 1, linha + 1, False)],
        # dyy de todas as regioes em 2 linhas
        [(dyy, x, y, True),(dyy, x, y, coluna, linha + 1, False)],
        [(dyy, x, y, coluna, linha + 1, True),(dyy, x, y, coluna + 1, linha + 1, False)],
    ]

    pols = np.array([])

    for item in funcoes_interna:
        if isinstance(item, tuple):
            pols = np.append(pols, getLineFromTuple(item))
        elif isinstance(item, list):
            pols = np.append(pols, getLineFromList(item))
    
    return np.reshape(pols, (-1, polinomio_size))

In [39]:

r = interno(1, 1)
print(r.shape)
print(r)

(16, 64)
[[ 1.  1.  1. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -6. -6. -6.]
 [ 0.  0.  2. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0. -2. -6.]]


# Obtendo a matriz de polinomios

In [43]:
def getMatrizPolinomio(matriz):
    linhas, colunas = matriz.shape

    pol = np.array([])
    for x in range(colunas):
        for y in range(linhas):
            if(column_edge(x, matriz) and line_edge(y, matriz)):
                pol = np.append(pol,vertice(x,y))
            elif(column_edge(x, matriz)):
                pol = np.append(pol,aresta_y(x,y))
            elif(line_edge(y, matriz)):
                pol = np.append(pol,aresta_x(x,y))
            else:
                pol = np.append(pol,interno(x,y))
    
    return np.reshape(pol, (-1, polinomio_size))

result = getMatrizPolinomio(matriz)
result.shape

(64, 64)

In [68]:
# print(matriz)
print(matriz.item(0,2))
print(np.flip(matriz, 0))

5
[[1 2 8]
 [1 4 3]
 [1 2 5]]
